In [11]:
### This will look for all the autoTMA images, and check if the corresponding cores are on the server
### if not, generate and upload them
## It also generate a composite core of the H&E and the TMA core

import dsaSecrets
import girder_client, os
%load_ext autoreload
%autoreload 2
import tmaHelpers as tma

gc = girder_client.GirderClient(apiUrl=dsaSecrets.apiUrl)
gc.authenticate(apiKey=dsaSecrets.apiToken)
## Samples Images 62bb03e4e70def4ae72689f5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


{'_id': '5cacfa86e62914004a60e139'}

In [13]:
tmaCoreInfo = {}  ### I'm going to stuck all of the information for all of the TMA slides into this dictionary for processing

### This will grab all the 
studyImageId = '62ded6cbe70def4ae726b228'
studyImages = list(gc.listItem(studyImageId))
for s in studyImages:
    autoTMAdata = getAutoTMAcoreInfo(gc,s['_id'])
    tmaCoreInfo[s['name']] = {'itemInfo': s, 'autoTMAdata': autoTMAdata}

2 autoTMA annotations found... will use the most recent one
150 total ROIs to export
1 autoTMA annotations found... will use the most recent one
150 total ROIs to export
1 autoTMA annotations found... will use the most recent one
45 total ROIs to export
1 autoTMA annotations found... will use the most recent one
150 total ROIs to export
1 autoTMA annotations found... will use the most recent one
45 total ROIs to export
1 autoTMA annotations found... will use the most recent one
150 total ROIs to export
0 autoTMA annotations found... will use the most recent one
No tma cores foudn for this image..
0 autoTMA annotations found... will use the most recent one
No tma cores foudn for this image..
0 autoTMA annotations found... will use the most recent one
No tma cores foudn for this image..
1 autoTMA annotations found... will use the most recent one
150 total ROIs to export


In [17]:
### get information for the TMA cores that are on the server..
tmaCorePath = '/collection/Badve_TMA_Project/StudyImages/autoTMAcores'
tcp = gc.get('resource/lookup?path=%s' % tmaCorePath)

tmaCoreImages = list(gc.listItem(tcp['_id']))
print(len(tmaCoreImages),'TMA Cores have been uploaded')

610 TMA Cores have been uploaded


In [19]:
gc.get('item/%s/tiles' % tmaCoreImages[2]['_id'])  ## This gets the individual tile info...

{'levels': 5,
 'magnification': 19.82553515625,
 'mm_x': 0.0005044000033889375,
 'mm_y': 0.0005044000033889375,
 'sizeX': 3040,
 'sizeY': 3616,
 'tileHeight': 256,
 'tileWidth': 256}

In [41]:
yamlFolder = '62e33fcde70def4ae726fcb7'
f = io.StringIO(yaml.dump(members))

gc.uploadFile(yamlFolder,f,'hellomoto.yaml',70,parentType='folder')

{'_id': '62e3424fe70def4ae726fcbb',
 '_modelType': 'file',
 'assetstoreId': '5ec45b61b3228813f0440a19',
 'created': '2022-07-29T02:13:35.411442+00:00',
 'creatorId': '5cacfa86e62914004a60e139',
 'exts': ['yaml'],
 'itemId': '62e3424fe70def4ae726fcba',
 'mimeType': 'application/octet-stream',
 'name': 'hellomoto.yaml',
 'size': 70}

In [39]:
# x = yaml.dump(members)
# len(x)
import io
len(yaml.dump(members))

70

In [20]:
import yaml

# dict object
members = [{'name': 'Zoey', 'occupation': 'Doctor'},
           {'name': 'Zaara', 'occupation': 'Dentist'}]

# Convert Python dictionary into a YAML document
print(yaml.dump(members))

- name: Zoey
  occupation: Doctor
- name: Zaara
  occupation: Dentist



In [57]:
def writeMultiCoreYaml( gc, imgIdOne, imgIdTwo):
    ### This will take two images and concanetate them together with imgOne on the left and image two on the right
    ### Using the nifty new YAML configuration...
    ### Get the tileInfo for these two slides..
    imgOneTI = gc.get('item/%s/tile')
    
    
    return (
    f'---\r\n'
    f'width: {7040}\r\n'
    f'height: {3000}]\r\n'
    'sources:\r\n'
    f'  - path: girder://{imgIdOne}\r\n'
    f'    z: 0\r\n'
    f'    position:\r\n'
    f'      x: 0\r\n'
    f'      y: 0\r\n'
    f'  - path: girder://{imgIdTwo}\r\n'
    f'    z: 0\r\n'
    f'    position:\r\n'
    f'      x: {imgIdOne}\r\n'
    f'      y: 0\r\n'
    )
    


In [58]:
f=writeMultiCoreYaml(gc,'123123','1231231')
print(f)

---
width: 7040
height: 3000]
sources:
  - path: girder://123123
    z: 0
    position:
      x: 0
      y: 0
  - path: girder://1231231
    z: 0
    position:
      x: 123123
      y: 0



In [9]:
def getAutoTMAcoreInfo( gc, imgId, tmaAnnotationName='autoTMA'):
    ### This will find and return the latest autoTMA record associated with a given image
    i = gc.get('item/%s' % imgId)
    fileRoot = i['meta']['slideNumber']
    
    ## Get autoTMA documents for this item
    tmaDocs=  gc.get('annotation?itemId=%s&name=%s' % (imgId, tmaAnnotationName))
    print(len(tmaDocs),'autoTMA annotations found... will use the most recent one')
    
    if tmaDocs:
        ae = gc.get('annotation/%s' % tmaDocs[-1]['_id']) ## Grab the last one... assume that's the most recent.. may need to check
        print(len(ae['annotation']['elements']),'total ROIs to export')
    else:
        ae = None
        print("No tma cores found for this image..")
    
    return ae
#     roiDir = 'ROI_Export'
    
#     if not os.path.isdir(roiDir):
#         os.mkdir(roiDir)

#     for roi in ae['annotation']['elements']:
#         roiFileName = "%s/%s.%s.%d.%d.tiff" % (roiDir,fileRoot,roi['label']['value'],roi['center'][0],roi['center'][1])
        
#         if not os.path.isfile(roiFileName):
#             print(roiFileName)
#             r= tma.get_annotation_region(gc,imgId,roi)        

#             with open(roiFileName,"wb") as fpi:
#                 fpi.write(r)
        
       